## Building Q&A application using Knowledge Bases for Amazon Bedrock - Retrieve API

### Context

In this notebook, we will dive deep into building Q&A application using Knowledge Bases for Amazon Bedrock - Retrieve API. Here, we will query the knowledge base to get the desired number of document chunks based on similarity search. We will then augment the prompt with relevant documents and query which will go as input to Anthropic Claude V2 for generating response.

With a knowledge base, you can securely connect foundation models (FMs) in Amazon Bedrock to your company
data for Retrieval Augmented Generation (RAG). Access to additional data helps the model generate more relevant,
context-speciﬁc, and accurate responses without continuously retraining the FM. All information retrieved from
knowledge bases comes with source attribution to improve transparency and minimize hallucinations. For more information on creating a knowledge base using console, please refer to this [post](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html).
We will cover 2 parts in the notebook:
- Part 1, we will share how you can use `RetrieveAPI` with foundation models from Amazon Bedrock. We will use the `mistral.mistral-7b` model. 
- Part 2, we will showcase the langchain integration using Anthropic Claude V2.1 model.

### Pattern

We can implement the solution using Retreival Augmented Generation (RAG) pattern. RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. Here, we are performing RAG effectively on the knowledge base created using console/sdk. 

### Pre-requisite

Before being able to answer the questions, the documents must be processed and ingested in vector database.

1. Load the documents into the knowledge base by connecting your s3 bucket (data source). 
2. Ingestion - Knowledge bases will split them into smaller chunks (based on the strategy selected), generate embeddings and store it in the associated vectore store and notebook [0_create_ingest_documents_test_kb.ipynb](./0\_create_ingest_documents_test_kb.ipynb) takes care of it for you.

![data_ingestion](./images/data_ingestion.png)


#### Notebook Walkthrough



For our notebook we will use the `Retreive API` provided by Knowledge Bases for Amazon Bedrock which converts user queries into
embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom
workﬂows on top of the semantic search results. The output of the `Retrieve API` includes the the `retrieved text chunks`, the `location type` and `URI` of the source data, as well as the relevance `scores` of the retrievals. 


We will then use the text chunks being generated and augment it with the original prompt and pass it through the `anthropic.claude-v2` model using prompt engineering patterns based on your use case.
    

### USE CASE:

#### Dataset

In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on. This data is already ingested into the Knowledge Bases for Amazon Bedrock. You will need the `knowledge base id` to run this example.
In your specific use case, you can sync different files for different domain topics and query this notebook in the same manner to evaluate model responses using the retrieve API from knowledge bases.


### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠

If you carry out the workshop from your local environment outside of the Amazon SageMaker studio please make sure you are running a Python runtime > 3.10.

### Setup

To run this notebook you would need to install following packages.


In [2]:
%pip install --upgrade pip
%pip install boto3==1.34.55 --force-reinstall --quiet
%pip install botocore==1.34.55 --force-reinstall --quiet
%pip install langchain==0.1.10 --force-reinstall --quiet

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.84 requires botocore==1.34.84, but you have botocore 1.34.121 which is incompatible.
opensearch-py 2.3.1 requires urllib3<2,>=1.21.1, but you have urllib3 2.2.1 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.84 requires botocore==1.34.84, but you have botocore 1.34.55 which is incompat

#### Restart the kernel with the updated packages that are installed through the dependencies above

In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
store -r kb_id

### Follow the steps below to initiate the bedrock client:

1. Import the necessary libraries, along with langchain for bedrock model selection, llama index to store the service context containing the llm and embedding model instances. We will use this service context later in the notebook for evaluating the responses from our Q&A application. 

2. Initialize `anthropic.claude-v2` as our large language model to perform query completions using the RAG pattern with the given knowledge base, once we get all text chunk searches through the `retrieve` API.

In [5]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-west-2


### Part 1 - Retrieve API with foundation models from Amazon Bedrock

Define a retrieve function that calls the `Retreive API` provided by Knowledge Bases for Amazon Bedrock which converts user queries into
embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom
workﬂows on top of the semantic search results. The output of the `Retrieve API` includes the the `retrieved text chunks`, the `location type` and `URI` of the source data, as well as the relevance `scores` of the retrievals. You can also use the  `overrideSearchType` option in `retrievalConfiguration` which offers the choice to use either `HYBRID` or `SEMANTIC`. By default, it will select the right strategy for you to give you most relevant results, and if you want to override the default option to use either hybrid or semantic search, you can set the value to `HYBRID/SEMANTIC`.

![retrieveAPI](./images/retrieveAPI.png)



In [6]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

#### Initialize your Knowledge base id before querying responses from the initialized LLM

Next, we will call the `retreive API`, and pass `knowledge base id`, `number of results` and `query` as paramters. 

`score`: You can view the associated score of each of the text chunk that was returned which depicts its correlation to the query in terms of how closely it matches it.

In [7]:
query = "What is Amazon doing in the field of generative AI?"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

[ { 'content': { 'text': 'This shift was driven by several factors, including '
                         'access to higher volumes of compute capacity at '
                         'lower prices than was ever available. Amazon has '
                         'been using machine learning extensively for 25 '
                         'years, employing it in everything from personalized '
                         'ecommerce recommendations, to fulfillment center '
                         'pick paths, to drones for Prime Air, to Alexa, to '
                         'the many machine learning services AWS offers (where '
                         'AWS has the broadest machine learning functionality '
                         'and customer base of any cloud provider). More '
                         'recently, a newer form of machine learning, called '
                         'Generative AI, has burst onto the scene and promises '
                         'to significantly accelerate machine

### Extract the text chunks from the retrieveAPI response

In the cell below, we will fetch the context from the retrieval results.

In [8]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [9]:
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

[ 'This shift was driven by several factors, including access to higher '
  'volumes of compute capacity at lower prices than was ever available. Amazon '
  'has been using machine learning extensively for 25 years, employing it in '
  'everything from personalized ecommerce recommendations, to fulfillment '
  'center pick paths, to drones for Prime Air, to Alexa, to the many machine '
  'learning services AWS offers (where AWS has the broadest machine learning '
  'functionality and customer base of any cloud provider). More recently, a '
  'newer form of machine learning, called Generative AI, has burst onto the '
  'scene and promises to significantly accelerate machine learning adoption. '
  'Generative AI is based on very Large Language Models (trained on up to '
  'hundreds of billions of parameters, and growing), across expansive '
  'datasets, and has radically general and broad recall and learning '
  'capabilities. We have been working on our own LLMs for a while now, believe

### Prompt specific to the model to personalize responses 

Here, we will use the specific prompt below for the model to act as a financial advisor AI system that will provide answers to questions by using fact based and statistical information when possible. We will provide the `Retrieve API` responses from above as a part of the `{contexts}` in the prompt for the model to refer to, along with the user `query`.  

In [10]:
prompt = f"""
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

### Invoke foundation model from Amazon Bedrock

In [11]:
# payload with model paramters
payload = json.dumps({
    "prompt": prompt,
    "max_tokens":512,
    "temperature":0.5,
    "top_k":50,
    "top_p":0.9
})

<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure you have subscribed to the <b>mistral-7B</b> model on the model access page in the Amazon Bedrock console before proceeding.
</div>

In [13]:
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
max_tokens = 512
top_p = 0.9
top_k = 50
temperature = 0.5

# Prompt with user turn only.
messages = [{"role": "user", "content":  prompt}]

In [16]:
body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages, 
            "temperature": temperature,
            "top_p": top_p, 
            "top_k": top_k
        }
    )

response = bedrock_client.invoke_model(body=body, modelId=model_id)
response_body = json.loads(response.get('body').read())
print(response_body['content'][0]['text'])

According to the context provided, Amazon has been investing heavily in generative AI based on large language models (LLMs). Some key points about Amazon's work in generative AI:

- Amazon has been working on its own large language models for a while and believes generative AI will "transform and improve virtually every customer experience." Amazon plans to continue investing substantially in these models across its consumer, seller, brand, and creator experiences.

- Like it has done with machine learning services on AWS, Amazon is working to democratize generative AI technology so companies of all sizes can leverage it. AWS is offering what it claims are the "most price-performant machine learning chips in Trainium and Inferentia" to enable small and large companies to train and run LLMs affordably.

- Amazon is delivering generative AI applications like CodeWhisperer on AWS, which uses an LLM to generate code suggestions for developers in real-time to boost productivity.

- The cont

## Part 2 - LangChain integration
In this notebook, we will dive deep into building Q&A application using Retrieve API provided by Knowledge Bases for Amazon Bedrock and LangChain. We will query the knowledge base to get the desired number of document chunks based on similarity search, integrate it with LangChain retriever and use Anthropic Claude V2.1 model for answering questions.

In [17]:
from langchain.llms.bedrock import Bedrock
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever

model_kwargs_claude = {
    "temperature": 0,
    "top_k": 10,
    "max_tokens_to_sample": 3000
}

llm = Bedrock(model_id="anthropic.claude-v2:1",
              model_kwargs=model_kwargs_claude,
              client = bedrock_client,)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Bedrock` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockLLM`.
  warn_deprecated(


Create a `AmazonKnowledgeBasesRetriever` object from LangChain which will call the `Retreive API` provided by Knowledge Bases for Amazon Bedrock which converts user queries into embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom workﬂows on top of the semantic search results. The output of the `Retrieve API` includes the the `retrieved text chunks`, the `location type` and `URI` of the source data, as well as the relevance `scores` of the retrievals.

In [20]:
query = "By what percentage did AWS revenue grow year-over-year in 2022?"
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 4,
                           'overrideSearchType': "SEMANTIC", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )
docs = retriever.invoke(
        input=query
    )
pp.pprint(docs)

[ Document(page_content='They estimate that, in 2020, third-party seller profits from selling on Amazon were between $25 billion and $39 billion, and to be conservative here I’ll go with $25 billion.   For customers, we have to break it down into consumer customers and AWS customers.   We’ll do consumers first. We offer low prices, vast selection, and fast delivery, but imagine we ignore all of that for the purpose of this estimate and value only one thing: we save customers time.   Customers complete 28% of purchases on Amazon in three minutes or less, and half of all purchases are finished in less than 15 minutes. Compare that to the typical shopping trip to a physical store – driving, parking, searching store aisles, waiting in the checkout line, finding your car, and driving home. Research suggests the typical physical store trip takes about an hour. If you assume that a typical Amazon purchase takes 15 minutes and that it saves you a couple of trips to a physical store a week, tha

## Prompt specific to the model to personalize responses
Here, we will use the specific prompt below for the model to act as a financial advisor AI system that will provide answers to questions by using fact based and statistical information when possible. We will provide the Retrieve API responses from above as a part of the `{context}` in the prompt for the model to refer to, along with the user `query`.

In [21]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
claude_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

### Integrating the retriever and the LLM defined above with RetrievalQA Chain to build the Q&A application.

In [22]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": claude_prompt}
)

In [23]:
answer = qa.invoke(query)
pp.pprint(answer)

{ 'query': 'By what percentage did AWS revenue grow year-over-year in 2022?',
  'result': ' <answer>\n'
            'The context states that AWS grew 29% year-over-year in 2022 on a '
            '$62B revenue base. So AWS revenue grew by 29% in 2022 compared to '
            'the previous year.\n'
            '</answer>',
  'source_documents': [ Document(page_content='They estimate that, in 2020, third-party seller profits from selling on Amazon were between $25 billion and $39 billion, and to be conservative here I’ll go with $25 billion.   For customers, we have to break it down into consumer customers and AWS customers.   We’ll do consumers first. We offer low prices, vast selection, and fast delivery, but imagine we ignore all of that for the purpose of this estimate and value only one thing: we save customers time.   Customers complete 28% of purchases on Amazon in three minutes or less, and half of all purchases are finished in less than 15 minutes. Compare that to the typical s

## Conclusion
You can use Retrieve API for customizing your RAG based application, using either `InvokeModel` API from Bedrock, or you can integrate with LangChain using `AmazonKnowledgeBaseRetriever`.
Retrieve API provides you with the flexibility of using any foundation model provided by Amazon Bedrock, and choosing the right search type, either HYBRID or SEMANTIC, based on your use case. 
Here is the [blog](#https://aws.amazon.com/blogs/machine-learning/knowledge-bases-for-amazon-bedrock-now-supports-hybrid-search/) for Hybrid Search feature, for more details.

<div class="alert alert-block alert-warning">
<b> Remember to CLEAN_UP at the end of your session. </b>
</div>